## Prompt Chaining 結合外部工具包括 Google 搜尋、計算機和股價API


In [29]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [51]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [30]:
import requests
import json
from pprint import pp

In [31]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, format_type="text"):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens, "response_format": { "type": format_type } }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## case1: Google search

OpenAI gpt-3.5-turbo 模型的 cut-off date 是 2021/9，最新 GPT-4-0125-preview 是 2023/12。因此沒有最新資料。

透過 google search 下來的數據，結合用戶問題，減少AI幻覺的可能性 

In [5]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python -q
# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from googlesearch import search

In [33]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

## 使用 google 官方的 search api 

In [15]:
pip install google-api-python -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google-api-python (from versions: none)
ERROR: No matching distribution found for google-api-python

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from googleapiclient.discovery import build
from urllib.parse import urlparse


google_api_key = os.getenv('google_api_key')

# 替換為您的 API 金鑰和搜索引擎 ID
API_KEY = google_api_key
SEARCH_ENGINE_ID = 'e25bf2d4283f04d7e'

def is_taiwan_domain(url):
    domain = urlparse(url).netloc
    return domain.endswith('.tw')

def google_search_from_google_api(keyword, num_results=5):
    service = build("customsearch", "v1", developerKey=API_KEY)

    res = service.cse().list(
        q=keyword,
        cx=SEARCH_ENGINE_ID,
        num=num_results * 2,  # 獲取更多結果以便過濾
        gl='tw',
        lr='lang_zh-TW',
        cr='countryTW'
    ).execute()

    results = []
    for item in res.get('items', []):
        link = item.get('link')
        result = {
            'title': item.get('title'),
            'link': link,
            'snippet': item.get('snippet')
        }
        results.append(result)
        if len(results) == num_results:
            break

    return results

if __name__ == "__main__":
    keyword = "nvidia 2024 年 股價表現"
    top_results = google_search_from_google_api(keyword, num_results=5)

    result_content = {}
    result_all = {}
    for idx, result in enumerate(top_results, start=1):

        result_content[f"標題"] = result['title']
        result_content[f"連結"] = result['link']
        result_content[f"簡介"] = result['snippet']

        result_all[f"資訊 {idx}"] = result_content

        result_content = {}

        # print(f"結果 {idx}:")
        # print("標題：", result['title'])
        # print("連結：", result['link'])
        # print("簡介：", result['snippet'])
        # print("-" * 40)

## 意圖偵測

何謂意圖偵測
顧名思義，意圖偵測就是要從使用者的輸入中找出使用者的意圖。在對話系統中，意圖偵測是非常重要的一環，因為只有知道使用者的意圖，才能提供正確的回應。

In [35]:
user_query = "請問 2024 年 Nvidia 的股價表現如何?"

prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)



'重要關鍵字: 2024年, Nvidia, 股價表現'


💡 AI 回覆有時候會多 label，例如上面的 "重要關鍵字"
這時候可以改一下 prompt 的結尾，加上引導文字，讓 AI 繼續接龍下去。

## 透過意圖偵測，找出關鍵字。並加上引導文字，丟入搜尋引擎中

In [36]:
prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
關鍵字:
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)

'2024年、Nvidia、股價、表現。'


In [37]:
search_result = google_search_from_google_api(keywords)
pp(search_result)

[{'title': '輝達股價大漲黃仁勳身價超越千億美元已可輕鬆買下整個英特爾',
  'link': 'https://tw.news.yahoo.com/%E8%BC%9D%E9%81%94%E8%82%A1%E5%83%B9%E5%A4%A7%E6%BC%B2-%E9%BB%83%E4%BB%81%E5%8B%B3%E8%BA%AB%E5%83%B9%E8%B6%85%E8%B6%8A%E5%8D%83%E5%84%84%E7%BE%8E%E5%85%83-%E5%B7%B2%E5%8F%AF%E8%BC%95%E9%AC%86%E8%B2%B7%E4%B8%8B%E6%95%B4%E5%80%8B%E8%8B%B1%E7%89%B9%E7%88%BE-045142531.html',
  'snippet': '2024年10月6日 ... 輝達在AI GPU領域的領先地位，促使其股價飆升。（攝影／張瀞文）. 根據 ... '
             '從輝達的股價表現來看，輝達在2024年10月的股價表現頗為波動。9月份\xa0...'},
 {'title': '輝達9月股價表現向來最差、客戶AI支出疑慮罩頂',
  'link': 'https://tw.stock.yahoo.com/news/%E8%BC%9D%E9%81%949%E6%9C%88%E8%82%A1%E5%83%B9%E8%A1%A8%E7%8F%BE%E5%90%91%E4%BE%86%E6%9C%80%E5%B7%AE-%E5%AE%A2%E6%88%B6ai%E6%94%AF%E5%87%BA%E7%96%91%E6%85%AE%E7%BD%A9%E9%A0%82-005500188.html',
  'snippet': '2024年9月3日 ... 摩根大通(JPMorgan Chase & Co.、通稱小摩)市場與投資策略董事長Michael '
             'Cembalest 2日發表研究報告指出，根據巴克萊估計，2024年打造的繪圖處理器(GPU)數量\xa0...'},
 {'title': 'AI前景無可限量？輝達市值突破3兆美元，成為全球第二大公司| 過去 ...',
  'link': 'https://global.udn.com/gl

In [42]:
prompt = f"""

請基於以下最新事實回答:

{str(search_result)}

用戶問題: {user_query}
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
print(response)

根據最新的報導，2024年Nvidia的股價表現相當驚人。在過去的幾個月中，Nvidia的股價一直呈現波動，但整體來說呈現上升趨勢。特別是在AI領域的表現讓投資者對Nvidia的未來充滿信心，這也反映在股價上。然而，也有一些報導指出，Nvidia的股價在某些時候可能會受到一些因素的影響而出現震盪。總的來說，Nvidia在2024年的股價表現仍然相當亮眼。


## case2: 外部計算機

(通常為兩個 prompt 組合: 抽取 -> 運算 -> 總結) </br>
生成式 AI 本質上屬於機率模型，不具備數學運算功能，必須借助外部工具才能達成任務

STEP1: 參數抽取

In [43]:
question = "請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234"

# prompt 參考自 langchain LLMMathChain
prompt = f"""
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
Expression: ${{single line mathematical expression that solves the problem}}

Begin.

Question: What is 37593 * 67?
Expression: 37593 * 67

Question: 37593^(1/5)
Expression: 37593**(1/5)

Question: {question}
Expression:

"""

In [44]:
pp(prompt)

('\n'
 'Translate a math problem into a expression that can be executed using '
 "Python's numexpr library. Use the output of running this code to answer the "
 'question.\n'
 '\n'
 'Question: ${Question with math problem.}\n'
 'Expression: ${single line mathematical expression that solves the problem}\n'
 '\n'
 'Begin.\n'
 '\n'
 'Question: What is 37593 * 67?\n'
 'Expression: 37593 * 67\n'
 '\n'
 'Question: 37593^(1/5)\n'
 'Expression: 37593**(1/5)\n'
 '\n'
 'Question: 請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234\n'
 'Expression:\n'
 '\n')


In [45]:
messages = [{"role": "user", "content": prompt }]

response = get_completion(messages, model="gpt-4")
pp(response)

'(64 * 2 - 8) / 100 ** 1.234'


STEP2: 運算

In [46]:
import numexpr

In [47]:
answer = numexpr.evaluate(response)

str(answer)

'0.40848982764120106'

STEP3: 總結

執行工具後的回傳，你可以選擇

1. 直接回給用戶 或是 轉成某種固定格式回給客戶，就不用再一次呼叫 LLM 了
3. 再丟給 LLM 變成自然語言再回給客戶


In [67]:
question

'請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234'

In [48]:
prompt = f"""
這是用戶問題: {question}

這其中的數學問題: {response} 和答案: {answer}

%%%
請用 JSON 格式回傳，遵照以下格式:
"answer": "string" // 答案

"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages, format_type="json_object")
pp(response)



'{\n    "answer": "0.40848982764120106"\n}'


In [49]:
eval(response)["answer"]

'0.40848982764120106'

## 案例三: 串接股價 API

這次我們要求 json 格式

### Step 1: 從用戶問題中，用 prompt1 來提取出 外部工具的參數

In [50]:
query = "請問113年的3月8號股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object" )
print(result)

data = json.loads(result)



{
  "error": "請提供日期和股票代號"
}


In [60]:
query = "請問 113年的10月30號的台積電2330，股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object", temperature=1)
print(result)

data = json.loads(result)

{
  "date": "20241030",
  "stock_code": "2330"
}


### Step 2: 執行工具，拿到結果

API 參考自 https://medium.com/%E5%B7%A5%E7%A8%8B%E9%9A%A8%E5%AF%AB%E7%AD%86%E8%A8%98/5%E7%A8%AE%E6%8A%93%E5%8F%96%E5%8F%B0%E8%82%A1%E6%AD%B7%E5%8F%B2%E8%82%A1%E5%83%B9%E7%9A%84%E6%96%B9%E6%B3%95-766bf2ed9d6

In [56]:
date = data["date"]
stock_code = data["stock_code"]

html = requests.get(f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock_code}')
context = json.loads(html.text)
context

{'stat': 'OK',
 'date': '20241030',
 'title': '113年10月 2330 台積電           各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['113/10/01',
   '27,877,267',
   '27,093,881,327',
   '967.00',
   '977.00',
   '967.00',
   '972.00',
   '+15.00',
   '37,862'],
  ['113/10/04',
   '43,765,757',
   '42,845,984,122',
   '986.00',
   '986.00',
   '967.00',
   '977.00',
   '+5.00',
   '43,935'],
  ['113/10/07',
   '43,850,831',
   '43,905,376,215',
   '993.00',
   '1,010.00',
   '989.00',
   '1,005.00',
   '+28.00',
   '79,851'],
  ['113/10/08',
   '35,344,059',
   '35,503,172,585',
   '1,000.00',
   '1,010.00',
   '997.00',
   '1,010.00',
   '+5.00',
   '42,227'],
  ['113/10/09',
   '53,208,610',
   '54,663,169,800',
   '1,030.00',
   '1,035.00',
   '1,020.00',
   '1,020.00',
   '+10.00',
   '80,835'],
  ['113/10/11',
   '47,776,351',
   '49,706,577,415',
   '1,025.00',
   '1,050.00',
   '1,020.00',
   '1,045.00',
   '+25.00',
   '89,915'],
  ['113/

In [57]:
import pandas as pd

df = pd.DataFrame(context["data"], columns=context["fields"])
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/10/01,"27,877,267","27,093,881,327",967.00,977.00,967.00,972.00,+15.00,"37,862"
1,113/10/04,"43,765,757","42,845,984,122",986.00,986.00,967.00,977.00,+5.00,"43,935"
2,113/10/07,"43,850,831","43,905,376,215",993.00,"1,010.00",989.00,"1,005.00",+28.00,"79,851"
3,113/10/08,"35,344,059","35,503,172,585","1,000.00","1,010.00",997.00,"1,010.00",+5.00,"42,227"
4,113/10/09,"53,208,610","54,663,169,800","1,030.00","1,035.00","1,020.00","1,020.00",+10.00,"80,835"
5,113/10/11,"47,776,351","49,706,577,415","1,025.00","1,050.00","1,020.00","1,045.00",+25.00,"89,915"
6,113/10/14,"39,906,157","41,745,442,865","1,045.00","1,055.00","1,035.00","1,045.00",0.00,"49,228"
7,113/10/15,"52,066,470","55,504,081,569","1,050.00","1,075.00","1,050.00","1,070.00",+25.00,"88,895"
8,113/10/16,"60,312,846","63,228,552,464","1,040.00","1,070.00","1,035.00","1,045.00",-25.00,"69,585"
9,113/10/17,"56,618,332","58,862,054,838","1,050.00","1,055.00","1,030.00","1,035.00",-10.00,"58,589"


### Step 3: 用 (prompt2 + 結果) 轉成自然語言回給用戶

In [61]:
messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

context: {context}
"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

根據資料顯示，113年10月30號的台積電2330股價表現如下：
- 開盤價：1,040.00
- 最高價：1,055.00
- 最低價：1,030.00
- 收盤價：1,030.00
- 漲跌價差：-10.00

這表示當天台積電2330的股價下跌了10.00點。
